In [1]:
import opendatasets as od
import numpy as np
import pandas as pd

In [10]:
from keras.models import Model
from keras.backend import shape,int_shape,random_normal,exp
from keras.layers import Input,Dense,Flatten,Conv2D,Conv2DTranspose,Lambda,Reshape,BatchNormalization,LeakyReLU,Dropout,Activation
from keras.preprocessing.image import ImageDataGenerator

In [3]:
od.download('https://www.kaggle.com/datasets/jessicali9530/celeba-dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading celeba-dataset.zip to .\celeba-dataset


100%|██████████| 1.33G/1.33G [28:10<00:00, 845kB/s]  


In [3]:
data_path = 'G:\Generative-deep-learning\Projects\celeb-face-vae\celeba-dataset\img_align_celeba'

In [4]:
input_dim = (128,128,3)
batch_size = 32

In [5]:
img_gen = ImageDataGenerator(rescale=1./255)
flow = img_gen.flow_from_directory(data_path, target_size=input_dim[:2], batch_size=batch_size,shuffle=True,
                                   class_mode='input',subset='training')

Found 202599 images belonging to 1 classes.


In [6]:
# network params
input_shape = input_dim
kernel_size = 3
batch_size = 32
layers = [32,64,64,64]
latent_dim = 200


In [7]:
input_layer = Input(shape=input_shape,name='encoder_input_layer')

x = input_layer

for layer in layers:
    conv_layer = Conv2D(filters=layer, kernel_size=3,strides=2, padding='same')
    x = conv_layer(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.5)(x)
    

shape_before_flatten = int_shape(x)

x = Flatten()(x)

mu = Dense(units=latent_dim, name = 'mu')(x)
log_var = Dense(units=latent_dim,name = 'log_var')(x)

encoder_mu_log_var =  Model(input_layer,(mu,log_var))

encoder_mu_log_var.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, 128, 128, 3  0          []                               
 r)                             )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   896         ['encoder_input_layer[0][0]']    
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [8]:
def sampling(args):
    mu,log_var = args
    epsilon = random_normal(shape=shape(mu), mean=0, stddev=1)
    return mu + exp(log_var/2)*epsilon

In [9]:
encoder_output = Lambda(sampling, name = 'encoder_output')([mu,log_var])

encoder = Model(input_layer,encoder_output)

encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, 128, 128, 3  0          []                               
 r)                             )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   896         ['encoder_input_layer[0][0]']    
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                            

In [24]:
# Decoder
decoder_temp_shape = np.prod(shape_before_flatten[1:])

decoder_input_layer = Input(shape=latent_dim,name='decoder_input')

x = Dense(decoder_temp_shape)(decoder_input_layer)
x = Reshape(shape_before_flatten[1:])(x)

for layer in layers[::-1]:
    conv_t_layer = Conv2DTranspose(filters=layer, kernel_size=3, strides=2, padding='same')
    x = conv_t_layer(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.5)(x)

decoder_output_layer = Activation('sigmoid')(x)

decoder = Model(decoder_input_layer,decoder_output_layer)